# S50F-DNN By Split  with Pipeline


Baseline Neural Network Model Performance



In [28]:
xseed=1982

from numpy.random import seed
seed(xseed)
from tensorflow import set_random_seed
set_random_seed(xseed)

In [29]:
 # Binary Classification with Sonar Dataset: Baseline
import numpy
import pandas


from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import models

from keras.utils import np_utils



from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.constraints import maxnorm


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline





from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import joblib

import os
from datetime import datetime

In [30]:
st='2007'
ed='2018'

strategyName='long'


In [31]:
filename='ML-Long_S50M15_07To1Q20-Train.csv'
filepath=os.path.abspath(f'D:\Dev-Trong\DataSet\/{filename}')
print(filepath)


D:\Dev-Trong\DataSet\ML-Long_S50M15_07To1Q20-Train.csv


# Train Data by WFA

In [32]:
# load dataset

dataframe = pandas.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)

dfOHLC=dataframe.loc[:,['open','high','low','close']]
dataframe.drop(['open','high','low','close'],axis=1,inplace=True)

                                                 
print(dataframe.shape)
              
dataframe.info()

(69679, 18)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 69679 entries, 2007-01-03 09:45:00 to 2020-03-31 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             69679 non-null  float64
 1   indy_ma-1100            69679 non-null  float64
 2   indy_hh-550             69679 non-null  float64
 3   indy_ll-550             69679 non-null  float64
 4   indy_mid-550            69679 non-null  float64
 5   indy_hh2-1100           69679 non-null  float64
 6   indy_ll2-1100           69679 non-null  float64
 7   indy_mid2-1100          69679 non-null  float64
 8   indy_macd110-440        69679 non-null  float64
 9   indy_signal110-440-110  69679 non-null  float64
 10  indy_hist_macd110-440   69679 non-null  float64
 11  indy_rsi25-ma20         69679 non-null  float64
 12  indy_6ATRTrail_DC-110   69679 non-null  float64
 13  cate_3trend-550_ma110   69679 non-null  int6

In [33]:
df=dataframe[st:ed]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62855 entries, 2007-01-03 09:45:00 to 2018-12-28 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             62855 non-null  float64
 1   indy_ma-1100            62855 non-null  float64
 2   indy_hh-550             62855 non-null  float64
 3   indy_ll-550             62855 non-null  float64
 4   indy_mid-550            62855 non-null  float64
 5   indy_hh2-1100           62855 non-null  float64
 6   indy_ll2-1100           62855 non-null  float64
 7   indy_mid2-1100          62855 non-null  float64
 8   indy_macd110-440        62855 non-null  float64
 9   indy_signal110-440-110  62855 non-null  float64
 10  indy_hist_macd110-440   62855 non-null  float64
 11  indy_rsi25-ma20         62855 non-null  float64
 12  indy_6ATRTrail_DC-110   62855 non-null  float64
 13  cate_3trend-550_ma110   62855 non-null  int64  
 14  cat

In [34]:
labelName=dataframe.columns[-1]
labelName

'LongSignal'

In [35]:
X_df = df.drop(columns=labelName).astype(float)
Y_df = df[labelName]
print(X_df.shape)
print(Y_df.shape)

(62855, 17)
(62855,)


In [36]:
# split into input (X) and output (Y) variables
X = X_df.to_numpy()
y =Y_df.to_numpy()
print(X.shape)
print(y.shape)

(62855, 17)
(62855,)


In [37]:
h1=17
h2=34
h3=17

xpoch=150
xbatch=32
#xpoch=10


In [38]:
def create_dnn_model1():

  model = Sequential()  
  
  model.add(Dense(h1, input_dim=X.shape[1], kernel_initializer='normal', W_regularizer=l2(0.00001),activation='relu' ))
  model.add(Dense(h2, kernel_initializer='normal', W_regularizer=l2(0.00001),activation='relu'))
  model.add(Dense(h3, kernel_initializer="normal",W_regularizer=l2(0.00001), activation="relu"))
  
  model.add(Dense(1, kernel_initializer='normal',W_regularizer=l2(0.00001), activation='sigmoid'))
  
    # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model  



In [39]:
def create_dnn_model2():
    
   
      
    model = Sequential()
    
    model.add(Dropout(0.2, input_shape=(X.shape[1],)))
    
    model.add(Dense(h1, kernel_initializer="normal", activation="relu", kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(h2, kernel_initializer="normal", activation="relu", kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(h3, kernel_initializer="normal", activation="relu", kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(1, kernel_initializer="normal", activation="sigmoid"))
    
    #adam_optimz = Adam(lr=0.05, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    

    learning_rate = 0.01
    decay_rate = learning_rate / xpoch
    momentum = 0.8
    sgd_optimz = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)    
   
    
    
    model.compile(loss="binary_crossentropy", optimizer=sgd_optimz , metrics=['accuracy'])

    
    #optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #optimizer = Adam(lr=0.05, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #optimizer = SGD(lr=0.001, momentum=0.9, decay=0.01, nesterov=True)
    #optimizer = Adadelta(lr=2.0, rho=0.99, epsilon=1e-08, decay=0.0)
    
    modelSelect=f'model2_LR{learning_rate}_mmt{momentum}'

    return model

In [40]:

#early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='auto')
estimator_steps=[]
estimator_steps.append(('standardize', StandardScaler()))
#estimator_steps.append(('minmax_range', MinMaxScaler()))

estimator_steps.append(('mlp', KerasClassifier(build_fn=create_dnn_model1, epochs=xpoch, batch_size=xbatch)))

mypipeline=Pipeline(estimator_steps)

In [41]:
mypipeline

Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x000001C41E11D648>)],
         verbose=False)

In [42]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=xseed)

In [43]:
mypipeline.fit(X_train,y_train)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(17, input_dim=17, kernel_initializer="normal", activation="relu", kernel_regularizer=<keras.reg...)`
  """
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(34, kernel_initializer="normal", activation="relu", kernel_regularizer=<keras.reg...)`
  
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(17, kernel_initializer="normal", activation="relu", kernel_regularizer=<keras.reg...)`
  import sys
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", activation="sigmoid", kernel_regularizer=<keras.reg...)`
  if __name__ == '__main__':



Epoch 1/150
42112/42112 [==============================] - 1s 31us/step - loss: 0.3336 - accuracy: 0.8582
Epoch 2/150
42112/42112 [==============================] - 1s 27us/step - loss: 0.2808 - accuracy: 0.8816
Epoch 3/150
42112/42112 [==============================] - 1s 26us/step - loss: 0.2595 - accuracy: 0.8908
Epoch 4/150
42112/42112 [==============================] - 1s 26us/step - loss: 0.2424 - accuracy: 0.8977
Epoch 5/150
42112/42112 [==============================] - 1s 26us/step - loss: 0.2285 - accuracy: 0.9055
Epoch 6/150
42112/42112 [==============================] - 1s 26us/step - loss: 0.2136 - accuracy: 0.9120
Epoch 7/150
42112/42112 [==============================] - 1s 26us/step - loss: 0.2026 - accuracy: 0.9151
Epoch 8/150
42112/42112 [==============================] - 1s 26us/step - loss: 0.1901 - accuracy: 0.9230
Epoch 9/150
42112/42112 [==============================] - 1s 27us/step - loss: 0.1811 - accuracy: 0.9279
Epoch 10/150
42112/42112 [===================

42112/42112 [==============================] - 1s 29us/step - loss: 0.0692 - accuracy: 0.9764
Epoch 75/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0674 - accuracy: 0.9767
Epoch 76/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0692 - accuracy: 0.9759
Epoch 77/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0691 - accuracy: 0.9759
Epoch 78/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0660 - accuracy: 0.9772
Epoch 79/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0651 - accuracy: 0.9773
Epoch 80/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0684 - accuracy: 0.9764
Epoch 81/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0658 - accuracy: 0.9775
Epoch 82/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0659 - accuracy: 0.9772
Epoch 83/150
42112/42112 [========================

42112/42112 [==============================] - 1s 29us/step - loss: 0.0415 - accuracy: 0.9866
Epoch 149/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0414 - accuracy: 0.9874
Epoch 150/150
42112/42112 [==============================] - 1s 28us/step - loss: 0.0417 - accuracy: 0.9879


Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x000001C41E11D648>)],
         verbose=False)

In [67]:
from sklearn.metrics import accuracy_score
# make predictions for test data and evaluate
y_pred = mypipeline.predict(X_test)
predictions = [value for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.26%


In [63]:
df_predictResult=pandas.DataFrame({'y_Actual':y_test,'y_Predicted':y_pred.ravel()})
df_predictResult.tail()

,y_Actual,y_Predicted
17068,1,1
17069,0,0
17070,0,0
17071,0,0
17072,0,0


In [64]:
from sklearn import metrics
pandas.DataFrame(metrics.confusion_matrix(y_test, predictions),columns=['Acutal_SideWay',f'Actual_{strategyName}'],index=['Pred_SideWay',f'Pred_{strategyName}'])

,Acutal_SideWay,Actual_long
Pred_SideWay,10531,31
Pred_long,96,6415


In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     10562
           1       1.00      0.99      0.99      6511

    accuracy                           0.99     17073
   macro avg       0.99      0.99      0.99     17073
weighted avg       0.99      0.99      0.99     17073



In [66]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[10531,    31],
       [   96,  6415]], dtype=int64)